In [16]:
from bs4 import BeautifulSoup
import json
import requests
import re

from time import sleep
from random import randint

import pandas as pd

from dateutil import parser
import sys

In [2]:
#Set to "False" to avoid updating data.
#Set to "True" to update, which can be a lengthy process.
update_general_data = True
update_specific_data = False

In [3]:
if update_general_data == False:
    try:
        TED_gen_df = pd.read_csv('TEDGeneral.csv', index_col=True)
    except:
        print('No general data to read, set "fetch general data" to "True" to aquire data')
        
if update_specific_data == False:
    try:
        pd.read_csv('TEDSpecific.csv')
    except:
        print('No specific data to read, set "fetch specific data" to "True" to aquire data')

No specific data to read, set "fetch specific data" to "True" to aquire data


In [4]:
def show_progress(part, whole):
    """
    Input:
    part = element of list
    whole = list
    ---------
    Function:
    Find the number of element "part" is within "whole"
    ---------
    Output:
    Return the string "[nth list element] of [list length]"
    """
    path = len(whole)
    step = whole.index(part) + 1
    progress = str(step) + " of " + str(path)
    return progress

In [5]:
def get_num_pages(start_page_soup):
    pagination_str = start_page_soup.find(class_ = 'pagination')
    page_num_list = []
    for child in pagination_str.contents:
        page_num = re.search(r'\d+', str(child))
        if page_num != None:
            page_num_list.append(int(page_num[0]))
    return max(page_num_list)

def build_page_urls(start_url, num_pages):
    url_list = []
    for i in range(1, num_pages + 1):
        url_list.append(re.sub(r'\d+', str(i), start_url))
    return url_list

In [6]:
def data_from_url(page_soup, data_dict):
    # return all talks to talk_data
    page_data = page_soup.find_all(class_ = 'talk-link')
    # iterate over results
    for talk in page_data:
        media_data = talk.find('span' , {'class':'thumb thumb--video thumb--crop-top'})
        message_data = talk.find(class_ = 'media__message')
        title_url_data = message_data.find(class_ = ' ga-link', attrs = {'data-ga-context': 'talks'})
        meta_data = message_data.find('div', {'class':'meta'})
        #get title
        talk_title = title_url_data.contents[0].strip()
        #get speaker
        talk_speaker = message_data.find(class_ = 'h12 talk-link__speaker').contents[0].strip()  
        #get url
        talk_url = 'https://www.ted.com' + title_url_data['href'].strip()
        #get date posted
        talk_date = meta_data.find(class_ = 'meta__item').find(class_ = 'meta__val').contents[0].strip()
        #get rated bool and rating
        if meta_data.find(class_ = 'meta__row'):
            talk_rated_bool = True
            talk_ratings = meta_data.find(class_ = 'meta__row').find(class_ = 'meta__val').contents[0].strip()
        else:
            talk_rated_bool = False
            talk_ratings = None
        #get duration
        talk_duration = media_data.find('span', {'class':'thumb__duration'}).contents[0].strip()
        data_dict[talk_url] = ({'title': talk_title, 'speaker': talk_speaker, 'date': talk_date,
                                      'rated_bool': talk_rated_bool, 'ratings': talk_ratings,
                                      'duration': talk_duration})
    return data_dict

In [7]:
if update_general_data == True:
    start_url = 'https://www.ted.com/talks?page=1'
    start_page = requests.get(start_url)
    start_page_soup = BeautifulSoup(start_page.text, 'html.parser')

    num_talk_pages = get_num_pages(start_page_soup)
    talks_url_list = build_page_urls(start_url, num_talk_pages)

In [8]:
if update_general_data == True:
    TED_talk_dict = {}
    for url in talks_url_list:
        sleep(randint(10,20))
        page = requests.get(url)
        if page.status_code != 200:
            print('We may have a problem here.', start_page.status_code)
            break

        page_soup = BeautifulSoup(page.text, 'html.parser')      
        TED_talk_dict = data_from_url(page_soup, TED_talk_dict)
        sys.stdout.write('\r'+ 'step ' + show_progress(url, talks_url_list))
    
    #make dataframe
    TED_gen_df = pd.DataFrame.from_dict(TED_talk_dict, orient='index').reset_index()
    
    #process dataframe
    TED_gen_df['url'] = TED_gen_df['index']
    TED_gen_df = TED_gen_df.drop('index', axis=1)
    
    TED_gen_df.sample(15)

step 77 of 77

ValueError: labels ['plays'] not contained in axis

In [9]:
if update_general_data == True:
    TED_gen_df.to_csv('TEDGeneral.csv', index=False)

<!DOCTYPE html>

<!--[if lt IE 8]> <html class="no-js loggedout oldie ie7" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js loggedout oldie ie8" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js loggedout" lang="en"><!--<![endif]-->
<head>
<script>
  (function (H){
  H.className=H.className.replace(/\bno-js\b/,'js');
  if (('; '+document.cookie).match(/; _ted_user_id=/)) H.className=H.className.replace(/\bloggedout\b/,'loggedin');
  })(document.documentElement)
</script><meta charset="utf-8"/>
<title>Sara Lewis: The loves and lies of fireflies | TED Talk</title>
<link href="https://www.ted.com/talks/sara_lewis_the_loves_and_lies_of_fireflies" rel="canonical"/>
<meta content="app-id=376183339,app-argument=https://www.ted.com/talks/sara_lewis_the_loves_and_lies_of_fireflies?utm_source=ted.com&amp;utm_medium=referral&amp;utm_campaign=smart_app_banner" name="apple-itunes-app">
<link href="https://www.ted.com/talks/sara_lewis_the_loves_and_lies_of_fireflies" hreflang

In [75]:
def extract_data_script(page_data, unique_start):
    page_str = str(page_data)
    start_idx = page_str.find(unique_start)
    num_script_start = re.finditer('<script', page_str)
    num_script_end = re.finditer('</script>', page_str)
    
    start_list = [s.start() for s in num_script_start]
    end_list = [e.end() for e in num_script_end]
    script_list = list(zip(start_list,end_list))
    
    narrowed_script_list = [s for s in script_list if s[0] >= start_idx ]
    
    if narrowed_script_list[0][1] < narrowed_script_list[1][0]:
        script_idx = narrowed_script_list[0]
    else:
        script_idx = -1
    
    data_extract = page_str[script_idx[0]:script_idx[1]]
    
    return data_extract
    
extracted_data = extract_data_script(talk_data, '<script>q("talkPage.init", ')

In [80]:
TED_gen_df['raw_data'] = '' 
for idx in TED_gen_df.index:
    page_url = TED_gen_df.iloc[idx]['url']
    talk_page = requests.get(page_url)
    
    sleep(randint(10,20))
    page = requests.get(url)
    if page.status_code != 200:
            print('We may have a problem here.', start_page.status_code)
            break
            
    talk_page_soup = BeautifulSoup(talk_page.text, 'html.parser')
    extracted_data = extract_data_script(talk_page_soup, '<script>q("talkPage.init", ')
    TED_gen_df.at[idx, 'raw_data'] = extracted_data
    sys.stdout.write('\r'+ 'step ' + show_progress(idx, list(TED_gen_df.index)))

step 417 of 2766

SSLError: HTTPSConnectionPool(host='www.ted.com', port=443): Max retries exceeded with url: /talks?page=77 (Caused by SSLError(SSLError("bad handshake: SysCallError(54, 'ECONNRESET')",),))

In [81]:
TED_gen_df

,title,speaker,date,rated_bool,ratings,duration,url,raw_data
0,"The mothers who found forgiveness, friendship",Aicha el-Wafi + Phyllis Rodriguez,May 2011,True,"Inspiring, Courageous",9:54,https://www.ted.com/talks/9_11_healing_the_mot...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
1,A choir as big as the Internet,Eric Whitacre,Apr 2010,True,"Beautiful, Inspiring",4:15,https://www.ted.com/talks/a_choir_as_big_as_th...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
2,My year of living biblically,AJ Jacobs,Jul 2008,True,"Funny, Fascinating",17:40,https://www.ted.com/talks/a_j_jacobs_year_of_l...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
3,A robot that flies like a bird,Markus Fischer,Jul 2011,True,"Ingenious, Fascinating",6:19,https://www.ted.com/talks/a_robot_that_flies_l...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
4,A TED speaker's worst nightmare,Colin Robertson,Mar 2012,True,"Funny, Ingenious",3:49,https://www.ted.com/talks/a_ted_speaker_s_wors...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
5,A whistleblower you haven't heard,Geert Chatrou,Feb 2011,True,"Beautiful, Fascinating",11:56,https://www.ted.com/talks/a_whistleblower_you_...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
6,"A dance in a hurricane of paper, wind and light",Aakash Odedra,Dec 2014,True,"Beautiful, Fascinating",9:50,https://www.ted.com/talks/aakash_odedra_a_danc...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
7,What it's like to be a parent in a war zone,Aala El-Khani,Feb 2017,True,"Informative, Inspiring",14:16,https://www.ted.com/talks/aala_el_khani_what_i...,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
8,America's native prisoners of war,Aaron Huey,Nov 2010,True,"Courageous, Informative",15:27,https://www.ted.com/talks/aaron_huey,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."
9,Visualizing ourselves ... with crowd-sourced data,Aaron Koblin,May 2011,True,"Fascinating, Inspiring",18:18,https://www.ted.com/talks/aaron_koblin,"<script>q(""talkPage.init"", {\n\t""el"": ""[data-t..."


In [82]:
TED_gen_df.to_csv('TEDData_part.csv', index=False)